<a href="https://colab.research.google.com/github/anisfreelancer71/mycode/blob/main/1_Model_Machine_Learning_Regression_Data_Harga_Rumah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Pemahaman Masalah (Problem Understanding):
1. Identifikasi dan definisikan masalah yang akan diselesaikan dengan machine learning.
2. Kumpulkan data yang relevan dan sesuai dengan masalah.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/filecsv"

 creditcard.csv		   'Salinan shopping_data.gsheet'
 data_harga_rumah.csv	    shopping_data.csv
 kapal_titanic.csv.gsheet   shopping_data.csv.gsheet
 lc_2016_2017.csv	    shopping_data.gsheet


#2. Pembersihan dan Eksplorasi Data (Data Cleaning and Exploration):

1. Analisis data yang ada untuk mengidentifikasi kecacatan dan ketidaksesuaian.
2. Lakukan pembersihan data untuk menghilangkan anomali dan missing values.
3. Eksplorasi data untuk memahami pola, korelasi, dan karakteristik data
4. Hapus fitur yang tidak relevan atau kurang informatif.

In [ ]:
import pandas as pd
import numpy as np
import math

data_rumah = pd.read_csv("/content/drive/My Drive/filecsv/data_harga_rumah.csv")

data_rumah.drop(['NAMA RUMAH'], axis=1, inplace=True)
kolom_hapus = [f'Unnamed: {i}' for i in range(8, 1001)]
data_rumah.drop(columns=kolom_hapus, inplace=True)

data_rumah.head()

,NO,HARGA,LB,LT,KT,KM,GRS
0,1,3800000000,220,220,3,3,0
1,2,4600000000,180,137,4,3,2
2,3,3000000000,267,250,4,4,4
3,4,430000000,40,25,2,2,0
4,5,9000000000,400,355,6,5,3


In [ ]:
data_rumah.describe()

,NO,HARGA,LB,LT,KT,KM,GRS
count,1010.000000,1.010000e+03,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000
mean,505.500000,7.628987e+09,276.539604,237.432673,4.668317,3.607921,1.920792
std,291.706188,7.340946e+09,177.864557,179.957604,1.572776,1.420066,1.510998
min,1.000000,4.300000e+08,40.000000,25.000000,2.000000,1.000000,0.000000
25%,253.250000,3.262500e+09,150.000000,130.000000,4.000000,3.000000,1.000000
50%,505.500000,5.000000e+09,216.500000,165.000000,4.000000,3.000000,2.000000
75%,757.750000,9.000000e+09,350.000000,290.000000,5.000000,4.000000,2.000000
max,1010.000000,6.500000e+10,1126.000000,1400.000000,10.000000,10.000000,10.000000


#3. Pemilihan Fitur (Feature Selection):

1. Identifikasi fitur (variabel) yang paling relevan untuk memprediksi atau mengklasifikasikan target.


In [ ]:
# Menghitung korelasi antara fitur dengan target (harga rumah)
korelasi = data_rumah.corr()['HARGA'].abs().sort_values(ascending=False)


In [ ]:
# Misalnya, Anda ingin memilih 5 fitur teratas berdasarkan korelasi
fitur_terpilih = korelasi[:6].index.tolist()

In [ ]:
# Menampilkan fitur yang terpilih
print("Fitur Terpilih:")
print(fitur_terpilih)

Fitur Terpilih:
['HARGA', 'LT', 'LB', 'GRS', 'KM', 'KT']


In [25]:
# Subset data hanya dengan fitur yang terpilih
data_rumah_terpilih = data_rumah[fitur_terpilih]
data_rumah_terpilih.head()

,HARGA,LT,LB,GRS,KM,KT
0,3800000000,220,220,0,3,3
1,4600000000,137,180,2,3,4
2,3000000000,250,267,4,4,4
3,430000000,25,40,0,2,2
4,9000000000,355,400,3,5,6


#4. Pembagian Data (Data Splitting):

1. Pisahkan data menjadi set pelatihan (training set), set validasi (validation set), dan set pengujian (test set).
2. Biasanya, data pelatihan digunakan untuk melatih model, data validasi digunakan untuk menyetel hiperparameter, dan data pengujian digunakan untuk mengevaluasi kinerja model.

In [27]:
from sklearn.model_selection import train_test_split

# Memisahkan fitur (X) dan target (y)
X = data_rumah_terpilih.drop('HARGA', axis=1)
y = data_rumah_terpilih['HARGA']

# Pembagian data: 70% pelatihan, 15% validasi, 15% pengujian
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Menampilkan ukuran set data yang terbagi
print(f"Jumlah data pelatihan: {len(X_train)}")
print(f"Jumlah data validasi: {len(X_val)}")
print(f"Jumlah data pengujian: {len(X_test)}")


Jumlah data pelatihan: 707
Jumlah data validasi: 151
Jumlah data pengujian: 152


#5. Pemilihan Model (Model Selection):

1. Pilih jenis model machine learning yang sesuai untuk masalah Anda (misalnya, regresi linier, pohon keputusan, jaringan saraf, dll.).
2. Mungkin perlu menguji beberapa model dan algoritma untuk memilih yang terbaik.

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Membuat objek model regresi linear
model_regresi = LinearRegression()

# Melatih model dengan data pelatihan
model_regresi.fit(X_train, y_train)

# Melakukan prediksi menggunakan data validasi
y_pred = model_regresi.predict(X_val)

# Mengukur kinerja model pada data validasi
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

# Menampilkan metrik evaluasi
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)


Mean Squared Error (MSE): 1.6370820105512366e+19
R-squared (R2): 0.6971885368988604


#6. Pelatihan Model (Model Training):

1. Latih model pada data pelatihan menggunakan algoritma yang dipilih.
2. Atur parameter dan hiperparameter model untuk meningkatkan kinerja.

##Penyetelan Hiperparameter:
Anda dapat mengatur beberapa hiperparameter dalam model regresi linear, seperti fit_intercept (apakah model harus memiliki intercept atau tidak), normalize (apakah fitur harus dinormalisasi), dan lain-lain. Misalnya:

In [35]:
from sklearn.linear_model import Ridge

model_ridge = Ridge(alpha=1.0)  # alpha adalah parameter regularisasi
model_ridge.fit(X_train, y_train)


Ridge()

#7. Validasi Model (Model Validation):

1. Evaluasi model pada data validasi untuk memastikan kinerjanya sesuai dengan harapan.
2. Lakukan penyetelan (tuning) lebih lanjut jika diperlukan.

In [36]:
from sklearn.metrics import mean_squared_error, r2_score

# Melakukan prediksi menggunakan data validasi yang sudah dinormalisasi
y_pred = model_regresi.predict(X_val_scaled)

# Mengukur kinerja model pada data validasi
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

# Menampilkan metrik evaluasi pada data validasi
print("Mean Squared Error (MSE) pada data validasi:", mse)
print("R-squared (R2) pada data validasi:", r2)

# Langkah Penyetelan Lanjutan (Model Tuning):
# Anda dapat melanjutkan dengan penyetelan hiperparameter atau perubahan model berdasarkan hasil evaluasi di atas.
# Contoh:
# 1. Penyetelan alpha pada Ridge Regression jika Anda menggunakan Ridge.
# 2. Menambahkan atau menghapus fitur-fitur yang tidak relevan.
# 3. Melakukan transformasi data jika dibutuhkan.
# 4. Memeriksa dan mengatasi overfitting atau underfitting.

# Setelah melakukan penyetelan, latih ulang model pada data pelatihan dengan hiperparameter yang baru
# dan evaluasi kinerja model yang diperbarui pada data validasi kembali.


Mean Squared Error (MSE) pada data validasi: 1.6370820105512362e+19
R-squared (R2) pada data validasi: 0.6971885368988605


#8. Evaluasi Model (Model Evaluation):

1. Gunakan data pengujian yang terpisah untuk mengukur kinerja model secara objektif.
2. Metrik evaluasi yang umum meliputi akurasi, presisi, recall, F1-score, dan lainnya, tergantung pada jenis masalah.

In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Normalisasi fitur-fitur pada data pengujian
X_test_scaled = scaler.transform(X_test)

# Melakukan prediksi menggunakan data pengujian yang sudah dinormalisasi
y_test_pred = model_regresi.predict(X_test_scaled)

# Mengukur kinerja model pada data pengujian
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Menampilkan metrik evaluasi pada data pengujian
print("Mean Squared Error (MSE) pada data pengujian:", mse_test)
print("R-squared (R2) pada data pengujian:", r2_test)


Mean Squared Error (MSE) pada data pengujian: 7.148660355277847e+18
R-squared (R2) pada data pengujian: 0.8326310547597301


#9. Optimisasi Model (Model Optimization):

Jika model tidak memenuhi kinerja yang diinginkan, lakukan perbaikan, seperti penambahan fitur baru, penggantian model, atau penyetelan parameter tambahan.

#10. Implementasi (Deployment):

1. Setelah model terlatih yang memadai ditemukan, siapkan model tersebut untuk produksi.
2. Buat antarmuka atau aplikasi yang memungkinkan pengguna untuk berinteraksi dengan model (misalnya, melalui API web).

In [ ]:
from flask import Flask, request, jsonify
import numpy as np

app = Flask(__name__)

# Prediksi harga rumah berdasarkan model yang sudah dilatih
def predict_price(features):
    # Normalisasi fitur input
    features_scaled = scaler.transform([features])

    # Melakukan prediksi harga
    predicted_price = model_regresi.predict(features_scaled)[0]

    return predicted_price

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Menerima data fitur dari pengguna
        data = request.json

        # Memeriksa apakah data sesuai dengan jumlah fitur yang diperlukan
        if len(data) != len(model_regresi.coef_):
            return jsonify({'error': 'Jumlah fitur tidak sesuai'}), 400

        # Melakukan prediksi harga
        predicted_price = predict_price(data)

        # Mengembalikan hasil prediksi sebagai JSON
        return jsonify({'predicted_price': predicted_price}), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


#11. Pemeliharaan dan Pemantauan (Maintenance and Monitoring):

Pemelihara model secara berkala dengan memperbarui model dengan data terbaru.
Pantau kinerja model di produksi dan tangani masalah atau perubahan dalam lingkungan dengan cepat.
#12. Optimasi Continu (Continuous Optimization):

Lanjutkan pemantauan dan pemeliharaan sambil mempertimbangkan perbaikan dan peningkatan model seiring waktu.